## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-28-23-51-07 +0000,nyt,Live Updates: Melissa Is Churning Toward Cuba ...,https://www.nytimes.com/live/2025/10/28/weathe...
1,2025-10-28-23-49-17 +0000,nypost,NFL Commissioner Roger Goodell hands out schol...,https://nypost.com/2025/10/28/us-news/nfl-comm...
2,2025-10-28-23-47-39 +0000,nypost,Thieves in reflector vests steal $3.2M from NY...,https://nypost.com/2025/10/28/us-news/thieves-...
3,2025-10-28-23-46-26 +0000,nypost,NYC ordering noise meters at construction site...,https://nypost.com/2025/10/28/us-news/nyc-orde...
4,2025-10-28-23-44-57 +0000,wapo,Police raid of Rio favelas leaves at least 64 ...,https://www.washingtonpost.com/world/2025/10/2...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2329/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
66,trump,50
135,new,27
2,melissa,20
46,hurricane,19
6,jamaica,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
240,2025-10-28-12-01-13 +0000,wapo,Japan’s new leader tries to charm Trump with g...,https://www.washingtonpost.com/politics/2025/1...,119
242,2025-10-28-11-49-55 +0000,nypost,Toyota pledges $10B toward new auto plants in ...,https://nypost.com/2025/10/28/us-news/trump-to...,108
300,2025-10-28-07-06-06 +0000,bbc,China controls the rare earths the world buys ...,https://www.bbc.com/news/articles/cj6ny24j0r3o...,106
96,2025-10-28-20-45-01 +0000,latimes,News Analysis: Trade deal or trade truce? Ques...,https://www.latimes.com/politics/story/2025-10...,102
172,2025-10-28-16-33-53 +0000,nypost,Mark Zuckerberg secretly met with AG Pam Bondi...,https://nypost.com/2025/10/28/business/mark-zu...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
240,119,2025-10-28-12-01-13 +0000,wapo,Japan’s new leader tries to charm Trump with g...,https://www.washingtonpost.com/politics/2025/1...
10,80,2025-10-28-23-36-57 +0000,nypost,Hurricane Melissa branded ‘storm of the centur...,https://nypost.com/2025/10/28/world-news/hurri...
180,59,2025-10-28-16-09-00 +0000,wsj,The Republican-led House Oversight Committee i...,https://www.wsj.com/politics/policy/republican...
141,54,2025-10-28-18-19-57 +0000,nypost,Foreign betting markets could influence NYC el...,https://nypost.com/2025/10/28/business/pro-mam...
21,49,2025-10-28-23-02-55 +0000,nypost,Airport delays are piling up as air traffic co...,https://nypost.com/2025/10/28/us-news/airport-...
89,47,2025-10-28-20-55-26 +0000,wapo,"Israel launches strikes on Gaza, claiming Hama...",https://www.washingtonpost.com/world/2025/10/2...
187,43,2025-10-28-15-26-21 +0000,nypost,Bill Gates makes major climate change reversal...,https://nypost.com/2025/10/28/us-news/bill-gat...
153,36,2025-10-28-17-19-06 +0000,cbc,"Texas sues Tylenol makers J&J and Kenvue, citi...",https://www.cbc.ca/news/health/texas-sues-tyle...
164,35,2025-10-28-16-56-21 +0000,nypost,More Americans want illegal immigrants booted ...,https://nypost.com/2025/10/28/us-news/even-dem...
326,32,2025-10-28-01-00-00 +0000,wsj,Watering down dish soap. Trading down to groce...,https://www.wsj.com/economy/consumers/the-leng...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
